<a href="https://colab.research.google.com/github/abaranguer/lab-aina/blob/main/xatbot_catal%C3%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# XatBot Català - GPT-J + AINA

In [1]:
!pip install ctranslate2 pyonmttok

In [2]:
import ctranslate2
import pyonmttok
from huggingface_hub import snapshot_download
from transformers import GPTJForCausalLM, AutoTokenizer
import torch

In [3]:
# AINA
class Translators:
  def __init__(self):
    model_dir_en_ca = snapshot_download(repo_id="projecte-aina/aina-translator-en-ca", revision="main", local_dir="/content/aina/en-ca")
    model_dir_ca_en = snapshot_download(repo_id="projecte-aina/aina-translator-ca-en", revision="main", local_dir="/content/aina/ca-en")

    self.tokenizerEnCa = pyonmttok.Tokenizer(mode="none", sp_model_path = model_dir_en_ca + "/spm.model")
    self.tokenizerCaEn = pyonmttok.Tokenizer(mode="none", sp_model_path = model_dir_ca_en + "/spm.model")

    self.tranlatorCa2En = ctranslate2.Translator(model_dir_ca_en)
    self.tranlatorEn2Ca = ctranslate2.Translator(model_dir_en_ca)

  def translateCa2En(self, text):
    tokenized = self.tokenizerCaEn.tokenize(text)
    translated = self.tranlatorCa2En.translate_batch([tokenized[0]])
    return self.tokenizerCaEn.detokenize(translated[0][0]['tokens'])

  def translateEn2Ca(self, text):
    tokenized = self.tokenizerEnCa.tokenize(text)
    translated = self.tranlatorEn2Ca.translate_batch([tokenized[0]])
    return self.tokenizerEnCa.detokenize(translated[0][0]['tokens'])

In [4]:
class XatBotGPTJ_AINA:
  def __init__(self, isLocal):
    self.translators = Translators()
    self.device = "cuda"

    model_save_dir = "/content/gpt-j-6B" # Define model_save_dir here

    self.model = None

    if not isLocal:
      # Load the model
      self.model = GPTJForCausalLM.from_pretrained(
        "EleutherAI/gpt-j-6B",
        revision="float16",
        torch_dtype=torch.float16,
      )
      # Save the model to a local directory
      self.model.save_pretrained(model_save_dir)
      self.tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")
    else:
      # Load the model from the local directory
      self.model = GPTJForCausalLM.from_pretrained(model_save_dir).to(self.device)
      # self.tokenizer = AutoTokenizer.from_pretrained(model_save_dir)
      self.tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")

  def xatBotSession(self):
    print("Hola, sóc el Xatbot GptJ-Aina.")
    print("Per a acabar la sessió, escriu 'Adéu'.")
    print("Parlem del que vulguis.")

    follow = True
    while follow:
      queryCat = input()
      if queryCat.upper() == "ADÉU":
        print("Ha estat un plaer parlar amb tu. Adéu!")
        follow = False
      else:
        query = self.translators.translateCa2En(queryCat)
        answerEn = self.getAnswer(query)
        answer = self.translators.translateEn2Ca(answerEn)
        print("XatBot:", answer)

  def getAnswer(self, query):
    input_ids = self.tokenizer("Q: " + query + "\nA:",return_tensors="pt").input_ids.to(self.device)

    gen_tokens = self.model.generate(
        input_ids,
        do_sample=True,
        temperature=0.9,
        max_length=1024)

    queryAndAnswer = self.tokenizer.batch_decode(gen_tokens, skip_special_tokens=True)[0]
    answerAndQuery = queryAndAnswer.split("A:")[1].strip()
    answer = answerAndQuery.split("Q:")[0].strip()

    return answer

In [5]:
if __name__ == '__main__':
  xatBot = XatBotGPTJ_AINA(True)
  xatBot.xatBotSession()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Hola, sóc el Xatbot GptJ-Aina.
Per a acabar la sessió, escriu 'Adéu'.
Parlem del que vulguis.
Bon dia, com estàs


...

XatBot: Aquest és el tipus de coses en què crec que m'hauria de centrar, perquè és el mateix una vegada i una altra, i no estic tan segur de poder millorar realment en això.
Ets conscient del que dius?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


XatBot: El que pensava sobre les meves paraules era que utilitzava la paraula “violació”. Però ara, aquesta paraula que he utilitzat pot sonar com un discurs d'odi. Però tot i així, he utilitzat la paraula “violació”.
Estàs molt locu!


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


XatBot: Ho estic. Estic tan boig.
Adéu
Ha estat un plaer parlar amb tu. Adéu!
